In [1]:
import pyspark

sc = pyspark.sql.SparkSession.Builder().getOrCreate()

## Levanto los archivos

In [2]:
import json

In [3]:
points_recep = sc.read.json('datos/points-recep.jsonlines').rdd

In [51]:
points_recep.filter(lambda x: x['recep_3'] != []).take(1)

[Row(Fecha='02/06/2018', Fin='16:58:40', Inicio='16:56:40', Punto=218, Tag='18', recep_0=[], recep_1=[], recep_2=[], recep_3=[91, 53], timestamp_fin='2018-06-02 16:58:40', timestamp_inicio='2018-06-02 16:56:40', x=463990.574953143, y=6111177.80221416)]

In [6]:
non_empty_points_recp = points_recep.filter(lambda x: len(x['recep_0']+x['recep_1']+x['recep_2']+x['recep_3']) > 0)


### Me quedo con una sola emision por punto

In [ ]:
unique_points = non_empty_points_recp.groupBy(lambda x: x['Punto']).map(lambda x: list(x[1])[0])

### Me quedo solo con puntos que tengan data de mas de una antena

In [42]:
#unique_points = unique_points.filter(lambda x: int(len(x['recep_0']) > 0) + int(len(x['recep_1']) > 0) + int(len(x['recep_2']) > 0) + int(len(x['recep_3']) > 0) > 1)



### Regresion por eje

In [43]:
import numpy as np

def generate_attrs(row):
    #f = lambda l: len(l)
    f = lambda l: float(np.mean(l) if len(l) > 0 else 0)
    data = {
        'recep_0': f(row['recep_0']),
        'recep_1': f(row['recep_1']),
        'recep_2': f(row['recep_2']),
        'recep_3': f(row['recep_3'])
    }
    target_x = row['x']
    target_y = row['y']
    return {'data': data, 'target_x': target_x, 'target_y': target_y}
    
points = unique_points.map(generate_attrs)

In [44]:
import pandas as pd

points_target_x = pd.DataFrame(points.map(lambda x: x['target_x']).collect())
points_target_y = pd.DataFrame(points.map(lambda x: x['target_y']).collect())
points_data = pd.DataFrame(points.map(lambda x: x['data']).collect())

In [45]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

def eval_knn(data, target):
    clf = KNeighborsRegressor(n_neighbors=5)
    scores = cross_val_score(clf, data, target, cv=5, scoring='neg_mean_absolute_error')
    print('cross val mae: {}'.format(scores))
    
print('prediccion eje x')
eval_knn(points_data, points_target_x)
print('prediccion eje y')
eval_knn(points_data, points_target_y)

prediccion eje x
cross val mae: [-165.48129858 -196.02377296 -369.80931374 -260.43770797 -289.79595259]
prediccion eje y
cross val mae: [-346.23796561 -232.58303659 -285.43055636 -199.53529696 -155.06426071]


### Clasificacion de puntos

In [46]:
def generate_attrs(row):
    #f = lambda l: len(l)
    f = lambda l: float(np.mean(l) if len(l) > 0 else 0)
    data = {
        'recep_0': f(row['recep_0']),
        'recep_1': f(row['recep_1']),
        'recep_2': f(row['recep_2']),
        'recep_3': f(row['recep_3'])
    }
    target = row['x']
    return {'data': data, 'target': row['Punto']}
    
points = unique_points.map(generate_attrs)


import pandas as pd

points_target = pd.DataFrame(points.map(lambda x: x['target']).collect())
points_data = pd.DataFrame(points.map(lambda x: x['data']).collect())



In [47]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold 
import math
dict_coordenadas = points_recep.map(lambda x: (x['Punto'],(x['x'], x['y']))).collectAsMap()
global dict_coordenadas

def distance(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)

def calculate_mae_distance(predictions, real):
    sum_error = 0
    for i in range(len(predictions)):
        pred_position = dict_coordenadas[predictions[i]]
        real_position = dict_coordenadas[real[i]]
        sum_error += distance(pred_position, real_position)
    return sum_error/len(predictions)
    

points_data_np = np.array(points_data)
points_target_np = np.array(points_target).ravel()
    


clf = KNeighborsClassifier(n_neighbors=5)
kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(points_data):
    X_train, X_test = points_data_np[train_index], points_data_np[test_index]
    y_train, y_test = points_target_np[train_index], points_target_np[test_index]
    
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    mae = calculate_mae_distance(predictions, y_test)
    print('mean absolute error {}'.format(mae))



mean absolute error 526.1846431709879
mean absolute error 398.336976494797
mean absolute error 499.22007681471473
mean absolute error 506.64639439536364
mean absolute error 511.8020749858015
